 
---

# Maximum Area Rectangle in a Binary Matrix

## 1. IP–OP–PS (Input, Output, Problem Statement)

### **Problem Statement**

Given a 2D binary matrix (containing only `0`s and `1`s), find the **maximum rectangular area** consisting entirely of `1`s. Each cell in the matrix can be treated as a bar of height 1 if the cell’s value is `1`; if the cell’s value is `0`, that bar’s height is effectively 0.

### **Input**

- Two integers, \(n\) (number of rows) and \(m\) (number of columns).
- An \(n \times m\) binary matrix (values are `0` or `1`).

### **Output**

- A single integer representing the **largest rectangular area** of `1`s in the matrix.

### **Detailed Example**

**Matrix** (4 rows, 5 columns):
```
[
  [1, 0, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [0, 1, 1, 0, 1],
  [1, 1, 1, 1, 1]
]
```

Possible largest rectangle of `1`s could be:
- The rectangle spanning rows 1 to 3 (0-based indexing) and columns 1 to 3, which has area = \(3 \times 3 = 9\).  
- Or another rectangle spanning multiple columns in a single row.  
- We systematically check each row as a histogram to find the maximum area rectangle.

For this example, the maximum area is often **9** (though it can vary based on the matrix’s layout).

---

## 2. Identification

### **Why Use the Stack Technique (Max Area Histogram)?**

1. **Link to Maximum Area Histogram**  
   - Each row of the binary matrix can be treated as a base.  
   - If we consider row \(i\), we accumulate “heights” of consecutive `1`s from row 0 up to row \(i\).  
   - Then, we apply the **Maximum Area Histogram** (MAH) approach to that row of heights.

2. **Key Cues**  
   - For each row, the brute force approach might scan the entire submatrix, leading to a large complexity.  
   - By reusing the known O(\(m\)) stack-based solution for each row, we achieve O(\(n \times m\)) overall.

3. **Characteristics**  
   - You only need to keep track of how many consecutive `1`s appear **vertically** up to the current row.  
   - Whenever you encounter a `0`, that height resets to 0.

---

## 3. Break Down → Row-by-Row Histogram Approach

### **Step-by-Step Sub-Tasks**

1. **Initialize a Height Array**  
   - Create an array `height[m]` (for \(m\) columns) to store consecutive vertical `1` counts.

2. **Process Each Row**  
   - For row \(i\) (from 0 to \(n-1\)):  
     - Update `height[j]`:
       - If `matrix[i][j] == 1`, then `height[j] += 1`.  
       - Else, `height[j] = 0`.  
     - Now, `height` represents a histogram for row \(i\).  
     - Compute **Max Area Histogram** (MAH) for this `height` array to get the largest rectangle in row 0..i.  
     - Track the global maximum area.

3. **Data Structures Used**  
   - A **stack** for the Max Area Histogram routine.  
   - An integer array `height` of length \(m\) to accumulate heights of consecutive `1`s.

4. **Overall Time Complexity**  
   - Each row calls the MAH function in O(\(m\)) time.  
   - There are \(n\) rows.  
   - Total time complexity = **O(\(n \times m\))**.

---

## 4. Explanations + Code

### **Detailed Explanation of Each Step**

1. **Building the Height Array**  
   - Initialize `height` to all zeros.  
   - For each row \(i\), update `height[j]` based on the current cell:
     \[
       \text{height}[j] = 
       \begin{cases}
       \text{height}[j] + 1 & \text{if } \text{matrix}[i][j] = 1\\
       0 & \text{otherwise}
       \end{cases}
     \]

2. **Max Area Histogram (MAH) on `height`**  
   - Use the well-known stack-based approach to find the largest rectangle in O(\(m\)) time.  
   - Maintain a **monotonically increasing** stack of bar indices:
     - When a bar of smaller height is encountered, pop from the stack and compute area for the popped bar.  
     - The area is `height_of_popped_bar * (current_index - stack_top_after_pop - 1)`.

3. **Keep Track of Global Maximum**  
   - After computing the max area for the histogram of row \(i\), compare it with a global maximum.  
   - The final global maximum after processing all rows is the answer.

### **C++ Implementation**

```cpp
#include <iostream>
#include <vector>
#include <stack>
using namespace std;

// Helper function: Maximum Area Histogram
int maxAreaHistogram(const vector<int> &heights) {
    int n = heights.size();
    stack<int> st;
    int maxArea = 0;

    for(int i = 0; i < n; i++) {
        // While current bar is smaller than top of stack, pop
        while(!st.empty() && heights[st.top()] > heights[i]) {
            int topIndex = st.top();
            st.pop();
            int leftBound = st.empty() ? -1 : st.top();
            int width = i - leftBound - 1;
            int area = heights[topIndex] * width;
            maxArea = max(maxArea, area);
        }
        st.push(i);
    }

    // Pop remaining bars
    for(int i = n; !st.empty(); i++) {
        int topIndex = st.top();
        st.pop();
        int leftBound = st.empty() ? -1 : st.top();
        int width = i - leftBound - 1;
        int area = heights[topIndex] * width;
        maxArea = max(maxArea, area);
    }

    return maxArea;
}

// Main function: Max Area Rectangle in a Binary Matrix
int maxAreaRectangleInBinaryMatrix(const vector<vector<int>> &matrix) {
    if(matrix.empty() || matrix[0].empty()) return 0;
    int n = matrix.size();
    int m = matrix[0].size();
    vector<int> height(m, 0);

    int maxRectArea = 0;

    for(int i = 0; i < n; i++) {
        // Update the height array
        for(int j = 0; j < m; j++) {
            if(matrix[i][j] == 1) {
                height[j] += 1;
            } else {
                height[j] = 0;
            }
        }
        // Compute max area for the current histogram
        int area = maxAreaHistogram(height);
        maxRectArea = max(maxRectArea, area);
    }

    return maxRectArea;
}

int main() {
    vector<vector<int>> matrix = {
        {1, 0, 1, 1, 1},
        {1, 1, 1, 1, 1},
        {0, 1, 1, 0, 1},
        {1, 1, 1, 1, 1}
    };

    int result = maxAreaRectangleInBinaryMatrix(matrix);
    cout << "Maximum area of rectangle of 1s: " << result << endl;
    return 0;
}
```

**Time Complexity:**  
- For each of the \(n\) rows, we run **Max Area Histogram** in O(\(m\)) → **O(\(n \times m\))** total.

---

## 5. Animated Visualization

Below is a **Python** snippet using `matplotlib` and `ipywidgets` to show how we build the `height` array row by row and compute the max area histogram. You can copy this into a Jupyter Notebook to visualize the algorithm’s internal state step by step.

```python
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def max_area_rect_visual(row_index):
    # Example binary matrix
    matrix = [
        [1, 0, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 0, 1],
        [1, 1, 1, 1, 1]
    ]
    n = len(matrix)
    m = len(matrix[0])
    
    # Height array up to the chosen row
    height = [0]*m
    
    # Build height array up to row_index
    for i in range(row_index+1):
        for j in range(m):
            if matrix[i][j] == 1:
                height[j] += 1
            else:
                height[j] = 0
    
    # Plot matrix rows up to row_index
    fig, ax = plt.subplots(1, 2, figsize=(12,4))
    ax[0].set_title(f"Binary Matrix Rows 0..{row_index}")
    for i in range(row_index+1):
        ax[0].plot(range(m), [i]*m, 'o', markersize=10,
                   markerfacecolor='white', markeredgecolor='black')
        for j in range(m):
            if matrix[i][j] == 1:
                ax[0].fill_between([j-0.4, j+0.4], i-0.4, i+0.4, color='blue', alpha=0.5)
    
    ax[0].invert_yaxis()  # so row 0 is at top
    ax[0].set_xlim(-0.5, m-0.5)
    ax[0].set_ylim(row_index+0.5, -0.5)
    ax[0].set_aspect('equal')
    ax[0].set_xticks(range(m))
    ax[0].set_yticks(range(row_index+1))
    
    # Show the histogram for row_index
    ax[1].bar(range(m), height, color='skyblue')
    ax[1].set_title(f"Histogram for row {row_index} (heights of consecutive 1s)")
    ax[1].set_xlabel("Column Index")
    ax[1].set_ylabel("Height of Bars")
    
    plt.tight_layout()
    plt.show()

# Create an interactive slider
interact(max_area_rect_visual, row_index=IntSlider(min=0, max=3, step=1, value=0));
```

### **How to Use This Visualization**

1. Copy this code into a Jupyter Notebook cell.  
2. Run the cell.  
3. Move the **slider** to select a row index.  
4. The left plot shows the submatrix (rows 0..row_index).  
5. The right plot shows the **height array** for that submatrix row, which we would then pass to the **max area histogram** function.  
6. This helps you see how the heights accumulate as we move down each row.

*(You could further extend the visualization to show the actual stack process of the Max Area Histogram, but the above snippet focuses on illustrating how the height array is built.)*

---

# Summary

1. **Problem Statement (IP–OP–PS):**  
   - We want to find the largest rectangle of `1`s in a 2D binary matrix.

2. **Identification:**  
   - Each row can be turned into a histogram of consecutive vertical `1`s.  
   - We reuse the **Max Area Histogram** approach for each row.

3. **Breakdown (Stack + Row-by-Row):**  
   - Maintain a `height` array of size \(m\).  
   - Update heights row by row.  
   - Apply the stack-based MAH algorithm on each updated row.

4. **Explanations + Code:**  
   - The final solution is O(\(n \times m\)).  
   - A full C++ implementation is provided.

5. **Animated Visualization:**  
   - The Python snippet shows how the `height` array is constructed per row, clarifying the approach step by step.
 

 

# Alternative Approach: Two-Pointer Method

## **High-Level Idea**

1. Maintain two pointers: **left** (start of the array) and **right** (end of the array).  
2. Keep track of the maximum height encountered so far from the **left** side (`leftMax`) and from the **right** side (`rightMax`).
3. Move one pointer inward each time, depending on which side is currently lower.  
4. Compute the trapped water based on how much the current bar is below the relevant max boundary.

---

## **Step-by-Step Explanation**

1. **Initialization**:
   - `left = 0`, `right = n - 1`
   - `leftMax = 0`, `rightMax = 0` (to store the highest bar seen so far from the left and right sides, respectively)
   - `totalWater = 0`

2. **While `left < right`**:
   - If `height[left] <= height[right]`:
     - If `height[left] >= leftMax`, update `leftMax = height[left]`.
       - (No water is trapped on top of this bar because it defines a new left boundary.)
     - Else, the bar is lower than `leftMax`; water trapped here = `leftMax - height[left]`.
     - Move the left pointer: `left++`.
   - Else (meaning `height[left] > height[right]`):
     - If `height[right] >= rightMax`, update `rightMax = height[right]`.
       - (No water is trapped on top of this bar because it defines a new right boundary.)
     - Else, water trapped here = `rightMax - height[right]`.
     - Move the right pointer: `right--`.

3. **Why it Works**:
   - Whichever side is lower (left or right), we know the water trapped on that side depends on how tall the boundary is from that same side (leftMax or rightMax).
   - If the left side is lower, we move `left` inward, because the right side is guaranteed to have a boundary that is at least as high as `height[right]`. The same logic applies in reverse if the right side is lower.
   - Each bar is processed once, making the algorithm O(n).

---

## **C++ Implementation (Two-Pointer Method)**

```cpp
#include <iostream>
#include <vector>
using namespace std;

int trapRainWaterTwoPointer(const vector<int> &height) {
    int n = height.size();
    if(n == 0) return 0;

    int left = 0, right = n - 1;
    int leftMax = 0, rightMax = 0;
    int totalWater = 0;

    while(left < right) {
        if(height[left] <= height[right]) {
            // If current left bar is taller than any we've seen, update leftMax
            if(height[left] >= leftMax) {
                leftMax = height[left];
            } else {
                // Otherwise, water can be trapped
                totalWater += (leftMax - height[left]);
            }
            left++;
        } else {
            // If current right bar is taller than any we've seen, update rightMax
            if(height[right] >= rightMax) {
                rightMax = height[right];
            } else {
                // Otherwise, water can be trapped
                totalWater += (rightMax - height[right]);
            }
            right--;
        }
    }
    return totalWater;
}

int main() {
    vector<int> heights = {0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1};
    int water = trapRainWaterTwoPointer(heights);
    cout << "Total water trapped (Two-Pointer): " << water << endl; // Expected output: 6
    return 0;
}
```

**Time Complexity**: O(n), since we move each pointer at most `n` steps (one pointer increments or decrements in each iteration).

**Space Complexity**: O(1), as we only use a few variables for tracking indices and max boundaries.

---

## **Comparing Stack vs. Two-Pointer Approaches**

1. **Stack Approach**:
   - Conceptually, it focuses on identifying “valleys” bounded by taller bars on both sides.  
   - Each index is pushed and popped at most once, leading to O(n) time and O(n) auxiliary space (for the stack in the worst case).

2. **Two-Pointer Approach**:
   - Maintains two boundaries (`leftMax` and `rightMax`) and moves inward.  
   - Also O(n) time, but O(1) auxiliary space.  
   - Often simpler to code once you understand the logic.

Both solutions are valid, efficient, and commonly taught. Choose based on which method you find more intuitive or if memory constraints favor the two-pointer approach.

---

# Full Summary

- **Stack Method**: We push indices of bars in decreasing order of heights. When we see a bar taller than the top of the stack, we pop and calculate trapped water using the new top as the left boundary.  
- **Two-Pointer Method**: We keep track of `leftMax` and `rightMax` from each side. Whichever side is lower, we move that pointer inward, because that side’s water depends on its own boundary.

Both approaches yield **O(n)** time complexity for the Rain Water Trapping problem.